2024-09-23 15:28:50.838 | INFO     | rocketshp.config:<module>:11 - PROJ_ROOT path is: /mnt/home/ssledzieski/Projects/rocketshp


In [2]:
import glob

import mdtraj as md
import nglview as nv
import numpy as np

In [4]:
pdb_files = glob.glob(f"./ensembles/{PDB_ID}/{PDB_ID}_*.pdb")

In [5]:
traj = md.load(pdb_files)

In [6]:
traj

<mdtraj.Trajectory with 49 frames, 2265 atoms, 307 residues, without unitcells at 0x7f6e3b3efe50>

In [7]:
from Bio.PDB import PDBParser

parser = PDBParser()

struct = parser.get_structure("A0QT50", "./ensembles/A0QT50_1/A0QT50_1_0.pdb")

In [8]:
fi_entropy = np.load("./ensembles/A0QT50_1/foldmason.fasta_3di.entropy.npy")

In [9]:
for c, bf in zip(struct[0]["A"].child_list, fi_entropy):
    for at in c.child_list:
        at.bfactor = 1 - bf

In [10]:
import statistics

from Bio.PDB import PDBParser, Residue, Structure
from matplotlib import colormaps
from matplotlib.colors import Normalize

CHAIN_COLORS = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]
AMINO_ACIDS = [
    "ALA",
    "ARG",
    "ASN",
    "ASP",
    "CYS",
    "GLN",
    "GLU",
    "GLY",
    "HIS",
    "ILE",
    "LEU",
    "LYS",
    "MET",
    "PHE",
    "PRO",
    "PYL",
    "SEC",
    "SER",
    "THR",
    "TRP",
    "TYR",
    "VAL",
]


def color_residues(structure: Structure.Structure, auth: bool = False) -> dict:
    _, resn, _ = zip(
        *[r.id for r in structure.get_residues() if r.get_resname() in AMINO_ACIDS]
    )

    rmin, rmax = min(resn), max(resn)
    # todo check for off by one errors
    norm = Normalize(vmin=rmin, vmax=rmax)
    auth_str = "_auth" if auth else ""

    cmap = colormaps["rainbow"]
    data = []
    for i in range(rmin, rmax):
        r, g, b, a = cmap(norm(i), bytes=True)
        color = {"r": int(r), "g": int(g), "b": int(b)}
        elem = {
            f"start{auth_str}_residue_number": i,
            f"end{auth_str}_residue_number": i,
            "color": color,
            "focus": False,
        }
        data.append(elem)

    color_data = {"data": data, "nonSelectedColor": None}
    return color_data

In [11]:
def get_bfactor(residue: Residue.Residue):
    """returns the residue-average b-factor"""
    return statistics.mean([atom.get_bfactor() for atom in residue])


def color_bfactor(structure: Structure.Structure, auth: bool = False) -> dict:
    auth_str = "_auth" if auth else ""
    value_data = []
    for chain in structure.get_chains():
        for r in chain.get_residues():
            if r.get_resname() in AMINO_ACIDS:
                bfactor = get_bfactor(r)
                elem = {
                    f"start{auth_str}_residue_number": r.id[1],
                    f"end{auth_str}_residue_number": r.id[1],
                    "struct_asym_id": chain.id,
                    "value": bfactor,
                }
                value_data.append(elem)

    all_values = [d["value"] for d in value_data]
    vmin, vmax = min(all_values), max(all_values)

    norm = Normalize(vmin=vmin, vmax=vmax)
    cmap = colormaps["inferno"]
    data = []
    for v_elem in value_data:
        elem = v_elem.copy()
        r, g, b, a = cmap(norm(elem.pop("value")), bytes=True)
        elem["color"] = {"r": int(r), "g": int(g), "b": int(b)}
        data.append(elem)

    color_data = {"data": data, "nonSelectedColor": None}
    return color_data

In [16]:
PDBeMolstar(molecule_id="2vda")

PDBeMolstar(bg_color='#F7F7F7', molecule_id='2vda')

In [12]:
from pathlib import Path

# fpth = Path("/afs/csail.mit.edu/u/s/samsl/Work/FoldNMA/alphaflow_pdb_base_202402/A0A172JI16_1.pdb")
fpth = Path("test_file.pdb")
struct = parser.get_structure("A0QT50", fpth)
custom_data = {
    "data": fpth.read_bytes(),
    "format": "pdb",
    "binary": False,
}
view = PDBeMolstar(
    custom_data=custom_data,
    color_data=color_bfactor(struct),
    hide_controls_icon=True,
    hide_expand_icon=True,
    hide_settings_icon=True,
    hide_selection_icon=True,
    hide_animation_icon=False,
    hide_water=True,
    hide_carbs=True,
)
view

PDBeMolstar(bg_color='#F7F7F7', color_data={'data': [{'start_residue_number': 1, 'end_residue_number': 1, 'str…

In [13]:
from Bio.PDB.PDBIO import PDBIO

io = PDBIO()

io.set_structure(struct)
io.save("test_file.pdb")

In [14]:
# view = nv.show_file("./ensembles/A0QT50_1/A0QT50_1_1.pdb")
view = nv.show_file("test_file.pdb")
view.remove_cartoon()
view.add_cartoon(selection="protein", color_scheme="bfactor")
view

NGLWidget()

In [15]:
view = nv.show_mdtraj(traj)
view.remove_cartoon()
view.add_cartoon("protein")
view

NGLWidget(max_frame=48)